In [3]:
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.DotNet.Interactive;
//using XPlot.Plotly.Interactive;
using System.Collections.Generic;
using System.IO;
using System;
using System.Text.RegularExpressions;

In [4]:
var fileExists = File.Exists("input.txt");
if (fileExists)
{
    display("This file exists.  Nothing has blown up.....yet.");
}

This file exists.  Nothing has blown up.....yet.

In [5]:
string[] lines = System.IO.File.ReadAllLines("input.txt");
lines.Length
//lines[0].Length


140

In [12]:
public class CharacterGrid
{
    public char[,] grid; // Internal 2D array for storing characters
    public int rows; // Number of rows in the grid
    public int columns; // Number of columns in the grid

    public CharacterGrid(int rows, int columns)
    {
        this.rows = rows;
        this.columns = columns;
        this.grid = new char[rows, columns];
    }

    // Get the character at a specific row and column
    public char GetCharacter(int row, int column)
    {
        if (!IsValidIndex(row, column))
        {
            return '.'; // return a useless '.' character if the index isn't valid
        }

        return grid[row, column];
    }

    // Set the character at a specific row and column
    public void SetCharacter(int row, int column, char character)
    {
        if (!IsValidIndex(row, column))
        {
            throw new IndexOutOfRangeException("Invalid row or column index");
        }

        grid[row, column] = character;
    }

    public void SetRow(string line, int rowNum)
    {
        int i = 0;
        foreach(char c in line)
        {
            SetCharacter(rowNum, i, c);
            i++;
        }
    }

    public List<PartNumber> GetUnverifiedParts(string line, int rowNum)
    {
        List<PartNumber> unverifiedParts = new List<PartNumber>();

        Regex regex = new Regex(@"(\d+)");

        foreach (Match match in regex.Matches(line))
        {
            int number = int.Parse(match.Groups[1].Value);
            int startPos = match.Index;
            int endPos = startPos + match.Length;
            PartNumber tempPart = new PartNumber(number, startPos, endPos, rowNum);
            unverifiedParts.Add(tempPart);
            //Console.WriteLine($"Line: {rowNum} Number: {number}, Start: {startPos}, End: {endPos}");
        }
        return unverifiedParts;
    }

    public List<PartNumber> GetVerifiedParts(List<PartNumber> unverifiedParts)
    {
        List<PartNumber> VerifiedParts = new List<PartNumber>();

        foreach (PartNumber pn in unverifiedParts)
        {
            bool isSpecial = false;
            foreach ((int row, int column) bc in pn.borderCharacters)
            {
                //Console.WriteLine("Looking for special character for value " + pn.value + " at position: " + bc.row + "/" + bc.column);
                bool actuallyIsSpecial = IsSpecialCharacter(bc.row, bc.column);
                if (actuallyIsSpecial)
                {
                    isSpecial = true;
                    break;
                }
            }
            if(isSpecial)
            {
                pn.isValid = true;
                VerifiedParts.Add(pn);
            }
            else
            {
                Console.WriteLine("Part value " + pn.value + " is not valid");
            }

        }
        return VerifiedParts;
    }

    public bool IsSpecialCharacter(int row, int column)
    {
        // looking for *, =, -, +, &, #, %, /, @, $
        char c = GetCharacter(row, column);
        bool special =  false;
        switch(c)
        {
            case '*':
            case '=':
            case '-':
            case '+':
            case '&':
            case '#':
            case '%':
            case '/':
            case '@':
            case '$':
                //Console.Out.WriteLine("Character " + c + " is special");
                special = true;
                break;
            default:
                special = false;
                break;
        }
        return special;

    }

    public List<(int row, int column)> GetGears(string[] grid)
    {
        List<(int row, int column)> positions = new List<(int row, int column)>();
        int gridRow = 0;
        foreach (string rowString in grid)
        {
            Regex regex = new Regex(@"\*");

            MatchCollection matches = regex.Matches(rowString);

            foreach (Match match in matches)
            {
                Console.WriteLine("row " + gridRow + "* position: " + match.Index);
                positions.Add((gridRow,match.Index));
            }
            gridRow++;
        }
        return positions;
    }

    // Check if a row and column index are within the grid bounds
    public bool IsValidIndex(int row, int column)
    {
        return row >= 0 && row < rows && column >= 0 && column < columns;
    }

    public bool HasValidIndicator()
    {
        return false;
    }


    
}

public class PartNumber
{
    public int value;
    public int startColumn;
    public int endColumn;
    public int row;
    public bool isValid;
    public List<(int row, int column)> borderCharacters;

    public PartNumber(int value, int startColumn, int endColumn, int row)
    {
        this.value = value;
        this.startColumn = startColumn;
        this.endColumn = endColumn-1;
        this.row = row;
        borderCharacters = GetBorderCharacters();
    }

    public List<(int row, int column)> GetBorderCharacters()
    {
        List<(int row, int column)> borderChars = new List<(int row, int column)>();
        int leftColumn = this.startColumn - 1;
        int rightColumn = this.endColumn +1;
        int topRow = this.row - 1;
        int bottomRow = this.row + 1;
        //Console.WriteLine($"TopRow: {topRow} BottomRow: {bottomRow}, LeftColumn: {leftColumn}, RightColumn: {rightColumn}");
        // Example - TopRow: 0 BottomRow: 2, LeftColumn: 2, RightColumn: 6
        // Add top and bottom rows.
        for (int i = leftColumn; i<=rightColumn; i++)
        {
            (int row, int column) temp = (topRow, i);
            (int row, int column) temp2 = (bottomRow, i);
            borderChars.Add(temp);
            borderChars.Add(temp2);
        }
        // Add middle row. Only add leftmost and rightmost columns.
        borderChars.Add((this.row, leftColumn));
        borderChars.Add((this.row, rightColumn));




        return borderChars;
    }

}



In [5]:
// Let's get a list of all characters, to specify the "special" ones.
List<char> allChars = lines.SelectMany(s => s.ToCharArray()).Distinct().ToList();

Console.WriteLine(string.Join(", ", allChars));

4, 8, ., 5, 0, 1, 3, 6, 2, 7, 9, *, =, -, +, &, #, %, /, @, $


In [21]:
CharacterGrid cg = new CharacterGrid(140, 140);
int i = 0;
List<PartNumber> parts = new List<PartNumber>();
foreach(string line in lines)
{
    cg.SetRow(line, i);
    List<PartNumber> tempParts = cg.GetUnverifiedParts(line, i);
    parts.AddRange(tempParts);
    i++;
}


//char x = cg.GetCharacter(137,136);

//bool isSpecial = cg.IsSpecialCharacter(137,136);

//PartNumber p = new PartNumber(48, 0, 1, 0);
//PartNumber p2 = new PartNumber(491,3, 5, 1);

//List<(int row, int column)> d = p2.GetBorderCharacters();

List<PartNumber> validParts = cg.GetVerifiedParts(parts);

int sum = 0;
foreach(PartNumber p in validParts)
{
    sum += p.value;
}

List<(int row, int column)> gearPositions = cg.GetGears(lines);

int gearSums = 0;

foreach ((int row, int column) gear in gearPositions)
{
    List<PartNumber> matches = new List<PartNumber>();
    foreach(PartNumber pm in parts)
    {
        // see if this gear matches a border character and add it to matches if it does.
        List<(int row, int column)> bcs = pm.borderCharacters;
        foreach((int row, int column) bc in bcs)
        {
            if(bc.row == gear.row && bc.column == gear.column)
            {
                matches.Add(pm);
                // don't add the same part multiple times
                break;
            }
        }
    }
    Console.WriteLine("Matches: " + matches.Count);
    if (matches.Count == 2)
    {
        int gearSum = matches[0].value * matches[1].value;
        gearSums += gearSum;
    }

    matches.Clear();
}
gearSums


Part value 48 is not valid
Part value 33 is not valid
Part value 622 is not valid
Part value 426 is not valid
Part value 843 is not valid
Part value 306 is not valid
Part value 597 is not valid
Part value 861 is not valid
Part value 864 is not valid
Part value 475 is not valid
Part value 729 is not valid
Part value 662 is not valid
Part value 414 is not valid
Part value 933 is not valid
Part value 726 is not valid
Part value 802 is not valid
Part value 474 is not valid
Part value 224 is not valid
Part value 615 is not valid
Part value 572 is not valid
Part value 995 is not valid
Part value 499 is not valid
Part value 574 is not valid
Part value 778 is not valid
Part value 236 is not valid
Part value 593 is not valid
Part value 36 is not valid
Part value 226 is not valid
Part value 790 is not valid
Part value 248 is not valid
Part value 173 is not valid
Part value 95 is not valid
Part value 877 is not valid
Part value 497 is not valid
Part value 427 is not valid
Part value 761 is not va

80694070

My initial answer of 491927 is wrong. However, I'm not sure what will make it right. It says that my answer (491927) is too low. I assume that my search for special characters in the border characters, so I'll need to debug that.
I was right about the border characters. The calculation was right but the end position was wrong, as evidenced by the sample data.
My new answer of 519543 is also too low, so I must be missing some other special character detection.

In [20]:
string[] sample = System.IO.File.ReadAllLines("sample.txt");

CharacterGrid cgs = new CharacterGrid(10, 10);


int i = 0;
List<PartNumber> parts = new List<PartNumber>();
foreach(string line in sample)
{
    cgs.SetRow(line, i);
    List<PartNumber> tempNumbers = cgs.GetUnverifiedParts(line, i);
    parts.AddRange(tempNumbers);
    i++;
}


//char x = cgs.GetCharacter(137,136);

//bool isSpecial = cgs.IsSpecialCharacter(137,136);

//PartNumber p = new PartNumber(48, 0, 1, 0);
//PartNumber p2 = new PartNumber(491,3, 5, 1);

//List<(int row, int column)> d = p2.GetBorderCharacters();

List<PartNumber> validParts = cgs.GetVerifiedParts(parts);

//validParts

int sum = 0;
foreach(PartNumber p in validParts)
{
    sum += p.value;
}

List<(int row, int column)> gearPositions = cgs.GetGears(sample);

int gearSums = 0;

foreach ((int row, int column) gear in gearPositions)
{
    List<PartNumber> matches = new List<PartNumber>();
    foreach(PartNumber pm in parts)
    {
        // see if this gear matches a border character and add it to matches if it does.
        List<(int row, int column)> bcs = pm.borderCharacters;
        foreach((int row, int column) bc in bcs)
        {
            if(bc.row == gear.row && bc.column == gear.column)
            {
                matches.Add(pm);
                // don't add the same part multiple times
                break;
            }
        }
    }
    Console.WriteLine("Matches: " + matches.Count);
    if (matches.Count == 2)
    {
        int gearSum = matches[0].value * matches[1].value;
        gearSums += gearSum;
    }

    matches.Clear();
}
gearSums




Part value 114 is not valid
Part value 58 is not valid
row 1* position: 3
row 4* position: 3
row 8* position: 5
Matches: 2
Matches: 1
Matches: 2


467835

The end of part one was looking at my debug of the parts and noticing that my regex was adding negative numbers.


Part two:

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

